
### This version does not use dask to load imagery


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
from osgeo import gdal, ogr, gdal_array # I/O image data
import numpy as np # math and array handling
import matplotlib.pyplot as plt # plot figures
from sklearn.ensemble import RandomForestClassifier # classifier
import pandas as pd # handling large data as table sheets
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix  # calculating measures for accuracy assessment

import seaborn as sn

import datetime

import joblib

In [2]:
import pickle

In [3]:
import os

print(os.environ['GDAL_NUM_THREADS'])
os.environ['PROJ_LIB'] = "/work/pi_gstuart_umass_edu/kate/conda/share/proj"

60


In [4]:
# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()

# define a number of trees that should be used (default = 500)
est = 500

# how many cores should be used?
# -1 -> all available cores
n_cores = 60

# the remote sensing image you want to classify
img_RS = r'/work/pi_gstuart_umass_edu/kate/allstacked_v1.tif'


# training and validation as shape files
training = r'/work/pi_gstuart_umass_edu/kate/OTH_Training_Data/OTH_Polygons_v3_relassed.shp'
validation = r'/work/pi_gstuart_umass_edu/kate/OTH_Training_Data/OTH_Polygons_v3_relassed.shp'

# what is the attributes name of your classes in the shape file (field name of the classes)?
attribute = 'ReClaSch A'


# directory, where the classification image should be saved:
classification_image = r'/work/pi_gstuart_umass_edu/kate/Classification_products/OTH_all_reclass_classification_v1.gtif'

# directory, where the all meta results should be saved:
results_txt = r'/work/pi_gstuart_umass_edu/kate/Classification_products/OTH_all_reclass_classification_v1.txt'

# laod training data and show all shape attributes

#model_dataset = gdal.Open(model_raster_fname)
shape_dataset = ogr.Open(training)
shape_layer = shape_dataset.GetLayer()

# extract the names of all attributes (fieldnames) in the shape file
attributes = []
ldefn = shape_layer.GetLayerDefn()
for n in range(ldefn.GetFieldCount()):
    fdefn = ldefn.GetFieldDefn(n)
    attributes.append(fdefn.name)
    
# print the attributes
print('Available attributes in the shape file are: {}'.format(attributes))

# prepare results text file:

print('Random Forest Classification', file=open(results_txt, "a"))
print('Processing: {}'.format(datetime.datetime.now()), file=open(results_txt, "a"))
print('-------------------------------------------------', file=open(results_txt, "a"))
print('PATHS:', file=open(results_txt, "a"))
print('Image: {}'.format(img_RS), file=open(results_txt, "a"))
print('Training shape: {}'.format(training) , file=open(results_txt, "a"))
print('Vaildation shape: {}'.format(validation) , file=open(results_txt, "a"))
print('      choosen attribute: {}'.format(attribute) , file=open(results_txt, "a"))
print('Classification image: {}'.format(classification_image) , file=open(results_txt, "a"))
print('Report text file: {}'.format(results_txt) , file=open(results_txt, "a"))
print('-------------------------------------------------', file=open(results_txt, "a"))

Available attributes in the shape file are: ['Transect', 'PointNum', 'SubClass', 'Northing', 'Easting', 'Altitude', 'Notes', 'Class', 'Raw Subcla', 'Pre Angle', 'Post Angle', 'mu', 'Class mu', 'Pre/Post', 'Hydro', 'ReClaSch A']


In [ ]:
# load image data
######THIS IS STEP THAT TAKES A LONG TIME#######

img_ds = gdal.Open(img_RS, gdal.GA_ReadOnly)

img = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount),
               gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))
print(img.shape, gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType), img.itemsize)
print(img_ds.GetRasterBand(1).ReadAsArray().itemsize)
for b in range(img.shape[2]):
    print(b)
    img[:, :, b] = img_ds.GetRasterBand(b + 1).ReadAsArray()
    
row = img_ds.RasterYSize
col = img_ds.RasterXSize
band_number = img_ds.RasterCount

print('Image extent: {} x {} (row x col)'.format(row, col))
print('Number of Bands: {}'.format(band_number))


print('Image extent: {} x {} (row x col)'.format(row, col), file=open(results_txt, "a"))
print('Number of Bands: {}'.format(band_number), file=open(results_txt, "a"))
print('---------------------------------------', file=open(results_txt, "a"))
print('TRAINING', file=open(results_txt, "a"))
print('Number of Trees: {}'.format(est), file=open(results_txt, "a"))

(12434, 13246, 142) <class 'numpy.float32'> 4
4
0
1


In [ ]:
# laod training data from shape file
# img_ds = gdal.Open(img_RS, gdal.GA_ReadOnly)

#model_dataset = gdal.Open(model_raster_fname)
shape_dataset = ogr.Open(training)
shape_layer = shape_dataset.GetLayer()

mem_drv = gdal.GetDriverByName('MEM')
mem_raster = mem_drv.Create('',img_ds.RasterXSize,img_ds.RasterYSize,1,gdal.GDT_UInt16)
#mem_raster.SetProjection(img_ds.GetProjection())
mem_raster.SetGeoTransform(img_ds.GetGeoTransform())
mem_band = mem_raster.GetRasterBand(1)
mem_band.Fill(0)
mem_band.SetNoDataValue(0)

att_ = 'ATTRIBUTE='+attribute
# http://gdal.org/gdal__alg_8h.html#adfe5e5d287d6c184aab03acbfa567cb1
# http://gis.stackexchange.com/questions/31568/gdal-rasterizelayer-doesnt-burn-all-polygons-to-raster
err = gdal.RasterizeLayer(mem_raster, [1], shape_layer, None, None, [1],  [att_,"ALL_TOUCHED=TRUE"])
assert err == gdal.CE_None

roi = mem_raster.ReadAsArray()

In [ ]:
# # Display images
# plt.subplot(121)
# plt.imshow(img[:, :, 0], cmap=plt.cm.Greys_r)
# plt.title('RS image - first band')

# plt.subplot(122)
# plt.imshow(roi, cmap=plt.cm.Spectral)
# plt.title('Training Image')

# plt.show()

# Number of training pixels:
n_samples = (roi > 0).sum()
print('{n} training samples'.format(n=n_samples))
print('{n} training samples'.format(n=n_samples), file=open(results_txt, "a"))

# What are our classification labels?
labels = np.unique(roi[roi > 0])
print('training data include {n} classes: {classes}'.format(n=labels.size, classes=labels))
print('training data include {n} classes: {classes}'.format(n=labels.size, classes=labels), file=open(results_txt, "a"))

# Subset the image dataset with the training image = X
# Mask the classes on the training dataset = y
# These will have n_samples rows
X = img[roi > 0, :]
y = roi[roi > 0]

print('Our X matrix is sized: {sz}'.format(sz=X.shape))
print('Our y array is sized: {sz}'.format(sz=y.shape))

In [ ]:
rf = RandomForestClassifier(n_estimators=est, oob_score=True, verbose=1, n_jobs=n_cores)

# verbose = 2 -> prints out every tree progression
# rf = RandomForestClassifier(n_estimators=est, oob_score=True, verbose=2, n_jobs=n_cores)



X = np.nan_to_num(X)
rf2 = rf.fit(X, y)


# Save the trained model to a file
pickle_file = '/work/pi_gstuart_umass_edu/kate/Classification_products/RF_alldata_v1.pkl'
with open(pickle_file, 'wb') as file:
    pickle.dump(rf2, file)

print ('model pickled')

# With our Random Forest model fit, we can check out the "Out-of-Bag" (OOB) prediction score:

print('--------------------------------', file=open(results_txt, "a"))
print('TRAINING and RF Model Diagnostics:', file=open(results_txt, "a"))
print('OOB prediction of accuracy is: {oob}%'.format(oob=rf.oob_score_ * 100))
print('OOB prediction of accuracy is: {oob}%'.format(oob=rf.oob_score_ * 100), file=open(results_txt, "a"))


# we can show the band importance:
bands = range(1,img_ds.RasterCount+1)

for b, imp in zip(bands, rf2.feature_importances_):
    print('Band {b} importance: {imp}'.format(b=b, imp=imp))
    print('Band {b} importance: {imp}'.format(b=b, imp=imp), file=open(results_txt, "a"))

    
# Let's look at a crosstabulation to see the class confusion. 
# To do so, we will import the Pandas library for some help:
# Setup a dataframe -- just like R
# Exception Handling because of possible Memory Error

try:
    df = pd.DataFrame()
    df['truth'] = y
    df['predict'] = rf.predict(X)

except MemoryError:
    print('Crosstab not available ')

else:
    # Cross-tabulate predictions
    print(pd.crosstab(df['truth'], df['predict'], margins=True))
    print(pd.crosstab(df['truth'], df['predict'], margins=True), file=open(results_txt, "a"))
    
# Predicting the rest of the image

# generate mask image from red band
mask = np.copy(img[:,:,0])
mask[mask > 0.0] = 1.0 # all actual pixels have a value of 1.0

# Take our full image and reshape into long 2d array (nrow * ncol, nband) for classification
old_shape = img.shape
new_shape = (img.shape[0] * img.shape[1], img.shape[2])
# img = img[:, :, :int(img.shape[2])].reshape(new_shape)
img = img.reshape(new_shape)

print('Reshaped from {o} to {n}'.format(o=old_shape, n=img.shape))

In [ ]:
# img = np.nan_to_num(img)
img[np.isnan(img)] = 0.0
# class_prediction = rf.predict(img)

print ("DONE!")


# **Apply Prediction**

In [ ]:

slices = int(round(len(img)/20))

test = True

while test == True:
    try:
        class_preds = list()

        temp = rf.predict(img[0:slices+1,:])
        class_preds.append(temp)

        for i in range(slices,len(img),slices):
            print('{} %, derzeit: {}'.format((i*100)/(len(img)), i))
            temp = rf.predict(img[i+1:i+(slices+1),:])                
            class_preds.append(temp)

    except MemoryError as error:
        slices = slices/4
        print('Not enought RAM, new slices = {}'.format(slices))

    else:
        test = False
else:
    print('Class prediction was successful without slicing!')
#concatenate all slices and re-shape it to the original extend
try:
    class_prediction = np.concatenate(class_preds,axis = 0)
except NameError:
    print('No slicing was necessary!')
    
class_prediction = class_prediction.reshape(old_shape[:2])
print('Reshaped back to {}'.format(class_prediction.shape))


# # generate mask image from red band
# mask = np.copy(img[:,:,0])
# mask[mask > 0.0] = 1.0 # all actual pixels have a value of 1.0

# plot mask

# plt.imshow(mask)

# mask classification an plot

class_prediction.astype(np.float16)
class_prediction_ = class_prediction*mask

cols = class_prediction.shape[1]
rows = class_prediction.shape[0]

class_prediction_.astype(np.float16)

driver = gdal.GetDriverByName("gtiff")
outdata = driver.Create(classification_image, cols, rows, 1, gdal.GDT_UInt16)
outdata.SetGeoTransform(img_ds.GetGeoTransform())##sets same geotransform as input
outdata.SetProjection(img_ds.GetProjection())##sets same projection as input
outdata.GetRasterBand(1).WriteArray(class_prediction_)
outdata.FlushCache() ##saves to disk!!
del outdata
print('Image saved to: {}'.format(classification_image))